# Clustering Crypto

In [96]:
# Initial imports
import requests
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from pathlib import Path

### Fetching Cryptocurrency Data

In [97]:
# Use the following endpoint to fetch json data
url = "https://min-api.cryptocompare.com/data/all/coinlist"

In [98]:
# Create a DataFrame 
# HINT: You will need to use the 'Data' key from the json response, then transpose the DataFrame.

In [99]:
# Alternatively, use the provided csv file:
file_path = Path("Resources/crypto_data.csv")
crypto_df = pd.read_csv(file_path, index_col=0)
crypto_df.head(10)

# Create a DataFrame

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
2015,2015 coin,X11,True,PoW/PoS,NaN,0
BTC,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,True,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,True,PoW,6.303924e+07,84000000


### Data Preprocessing

In [100]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'

crypto_df[100:134]

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
GLC,GlobalCoin,Scrypt,True,PoW,6.567272e+07,70000000
GLD,GoldCoin,Scrypt,True,PoW,4.165865e+07,72245700
GLX,GalaxyCoin,Scrypt,True,PoW/PoS,NaN,538214400
GLYPH,GlyphCoin,X11,True,PoW/PoS,0.000000e+00,7000000
GML,GameLeagueCoin,Scrypt,True,PoW/PoS,NaN,500000000
GUE,GuerillaCoin,X11,True,PoW/PoS,NaN,19500000
HAL,Halcyon,X15,True,PoW/PoS,0.000000e+00,0
HBN,HoboNickels,Scrypt,True,PoW/PoS,8.926693e+07,120000000
HUC,HunterCoin,Multiple,True,PoW,0.000000e+00,42000000
HVC,HeavyCoin,Multiple,True,PoW,NaN,128000000


In [101]:
# Keep only cryptocurrencies that are trading
crypto_df = crypto_df[crypto_df['IsTrading'] == True] 

In [102]:
# Keep only cryptocurrencies with a working algorithm
crypto_df[100:134]

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
GLD,GoldCoin,Scrypt,True,PoW,4.165865e+07,72245700
GLX,GalaxyCoin,Scrypt,True,PoW/PoS,NaN,538214400
GLYPH,GlyphCoin,X11,True,PoW/PoS,0.000000e+00,7000000
GML,GameLeagueCoin,Scrypt,True,PoW/PoS,NaN,500000000
GUE,GuerillaCoin,X11,True,PoW/PoS,NaN,19500000
HAL,Halcyon,X15,True,PoW/PoS,0.000000e+00,0
HBN,HoboNickels,Scrypt,True,PoW/PoS,8.926693e+07,120000000
HUC,HunterCoin,Multiple,True,PoW,0.000000e+00,42000000
HVC,HeavyCoin,Multiple,True,PoW,NaN,128000000
HYP,HyperStake,X11,True,PoS,1.678520e+09,0


In [103]:
# Remove the "IsTrading" column
crypto_df.drop('IsTrading',axis = 1, inplace = True)
crypto_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...
SERO,Super Zero,Ethash,PoW,NaN,1000000000
UOS,UOS,SHA-256,DPoI,NaN,1000000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [104]:
# Remove rows with at least 1 null value
crypto_df = crypto_df.dropna(axis=0, how="any")
crypto_df.head(20)

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,PoW,6.303924e+07,84000000
DASH,Dash,X11,PoW/PoS,9.031294e+06,22000000
XMR,Monero,CryptoNight-V7,PoW,1.720114e+07,0
ETC,Ethereum Classic,Ethash,PoW,1.133597e+08,210000000


In [105]:
# Remove rows with cryptocurrencies having no coins mined
crypto_df = crypto_df[crypto_df["TotalCoinsMined"] > 0]
crypto_df.head(20)

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,PoW,6.303924e+07,84000000
DASH,Dash,X11,PoW/PoS,9.031294e+06,22000000
XMR,Monero,CryptoNight-V7,PoW,1.720114e+07,0
ETC,Ethereum Classic,Ethash,PoW,1.133597e+08,210000000
ZEC,ZCash,Equihash,PoW,7.383056e+06,21000000


In [106]:
# Drop rows where there are 'N/A' text values
crypto_df = crypto_df[crypto_df.iloc[:] != 'N/A'].dropna()
crypto_df.shape

(532, 5)

In [107]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
crypto_coinname = pd.DataFrame(crypto_df['CoinName'],index=crypto_df.index)
crypto_coinname

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum
...,...
ZEPH,ZEPHYR
GAP,Gapcoin
BDX,Beldex
ZEN,Horizen


In [108]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
crypto_df.drop('CoinName', axis=1, inplace=True)
crypto_df

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0
...,...,...,...,...
ZEPH,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Equihash,PoW,7.296538e+06,21000000


In [109]:
# Create dummy variables for text features
X = pd.get_dummies(crypto_df, columns = ['Algorithm','ProofType'] )
X                                         

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZEPH,2.000000e+09,2000000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GAP,1.493105e+07,250000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BDX,9.802226e+08,1400222610,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ZEN,7.296538e+06,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [110]:
# Standardize data
scaler = StandardScaler()
scaled_crypto = StandardScaler().fit_transform(X)
scaled_crypto

array([[-0.11710817, -0.1528703 , -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.09396955, -0.145009  , -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [ 0.52494561,  4.48942416, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       ...,
       [-0.09561336, -0.13217937, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.11694817, -0.15255998, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.11710536, -0.15285552, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ]])

### Reducing Dimensions Using PCA

In [111]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA(n_components=3)

In [112]:
# Create a DataFrame with the principal components data
crypto_pca = pca.fit_transform(scaled_crypto)
crypto_pca

array([[-0.33347604,  1.02240465, -0.56359256],
       [-0.31680017,  1.022455  , -0.56410524],
       [ 2.30206255,  1.68913531, -0.72280272],
       ...,
       [ 0.32234781, -2.30738714,  0.3648853 ],
       [-0.16800599, -1.99788307,  0.43624596],
       [-0.2880046 ,  0.79069978, -0.2124666 ]])

In [113]:
pcs_df = pd.DataFrame(crypto_pca, columns=["PC 1", "PC 2", "PC 3"], index=crypto_df.index)
pcs_df

,PC 1,PC 2,PC 3
42,-0.333476,1.022405,-0.563593
404,-0.316800,1.022455,-0.564105
1337,2.302063,1.689135,-0.722803
BTC,-0.150501,-1.310736,0.214456
ETH,-0.150998,-2.032786,0.404804
...,...,...,...
ZEPH,2.457385,0.887287,0.134812
GAP,-0.331520,1.022277,-0.563644
BDX,0.322348,-2.307387,0.364885
ZEN,-0.168006,-1.997883,0.436246


In [114]:
## Fetch the explained variance
pca.explained_variance_ratio_

array([0.02793097, 0.02141428, 0.0204884 ])

### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [115]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)

# Create the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")



:Curve   [k]   (inertia)

Running K-Means with `k=<your best value for k here>`

In [118]:
# Initialize the K-Means model
model = KMeans(n_clusters=3, random_state=0)

# Fit the model
model.fit(pcs_df)

# Predict clusters
predicted = model.predict(pcs_df)

# Create a new DataFrame including predicted clusters and cryptocurrencies features
clustered_df = crypto_df.join(pcs_df)
clustered_df['CoinName'] = crypto_coinname['CoinName']
clustered_df['Class'] = predicted
clustered_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,Class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.333476,1.022405,-0.563593,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.316800,1.022455,-0.564105,404Coin,0
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.302063,1.689135,-0.722803,EliteCoin,0
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.150501,-1.310736,0.214456,Bitcoin,2
ETH,Ethash,PoW,1.076842e+08,0,-0.150998,-2.032786,0.404804,Ethereum,2


### Visualizing Results

#### 3D-Scatter with Clusters

In [124]:
# Create a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clustered_df,
    x="PC 1",
    y="PC 2",
    z="PC 3",
    color="Class",
    symbol="Class",
    width=800,
    hover_name="CoinName",
    hover_data=["Algorithm"]
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()

#### Table of Tradable Cryptocurrencies

In [125]:
# Table with tradable cryptos
# The table should have the following columns: 
# "CoinName", "Algorithm", "ProofType", "TotalCoinSupply", "TotalCoinsMined", "Class"
clustered_df.hvplot.table(
    columns=['CoinName','Algorithm', 'ProofType', 'TotalCoinSupply', 'TotalCoinsMined', 'Class'], 
    sortable=True, 
    selectable=True
)


:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

In [130]:
# Print the total number of tradable cryptocurrencies

print(f"Total # of Tradable Cryptocurrencies: {(len(clustered_df.CoinName))}")

Total # of Tradable Cryptocurrencies: 532


#### Scatter Plot with Tradable Cryptocurrencies

In [132]:
# Scale data to create the scatter plot
pd.set_option('display.float_format', lambda x: '%.5f' % x)
clustered_df.head()


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,Class
42,Scrypt,PoW/PoS,41.99995,42,-0.33348,1.02240,-0.56359,42 Coin,0
404,Scrypt,PoW/PoS,1055184902.04000,532000000,-0.31680,1.02245,-0.56411,404Coin,0
1337,X13,PoW/PoS,29279424622.50270,314159265359,2.30206,1.68914,-0.72280,EliteCoin,0
BTC,SHA-256,PoW,17927175.00000,21000000,-0.15050,-1.31074,0.21446,Bitcoin,2
LTC,Scrypt,PoW,63039243.30000,84000000,-0.16197,-1.14306,0.00529,Litecoin,2


In [178]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
# Create a scatter plot using hvplot.scatter
# Use the hover_cols=["CoinName"] so each data point has a name


from bokeh.models.formatters import BasicTickFormatter

clustered_df.hvplot.scatter(
    x="TotalCoinsMined",
    y="TotalCoinSupply",
    hover_cols=["CoinName"],
    by="Class",
    width=700,
    height=2500,
    scale=10000000,
    xformatter='%.0f',
    yformatter='%.0f'
)


:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)